<a href="https://colab.research.google.com/github/MiyoBran/Alura-ONE-G9/blob/main/formacion-Aprendiendo-a-hacer-ETL-G9-ONE/02-pandas-e-s-diferentes-formatos-archivo/ETL_02_Pandas_I_O_Banco_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Aula 5 - Leyendo banco de datos**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Creando un banco de datos local**

In [2]:
import sqlalchemy

In [3]:
from sqlalchemy import create_engine, MetaData, Table, inspect, text

In [4]:
engine = create_engine('sqlite:///:memory:')

### 🗄️ Motores de Bases de Datos (SQLAlchemy y SQLite)

**Concepto de Negocio:**
Mientras que los archivos CSV o JSON son excelentes para transferir datos, las Bases de Datos Relacionales (SQL) son el motor transaccional donde viven los datos operativos de una empresa. Permiten cruzar información de múltiples tablas (JOINs), filtrar millones de registros en milisegundos y mantener la integridad de los datos.

**La Arquitectura (SQLAlchemy):**
Python no habla el idioma SQL de forma nativa. `SQLAlchemy` es la librería estándar de la industria (un ORM / Database Toolkit) que actúa como puente universal. Nos permite escribir código en Pandas y que SQLAlchemy se encargue de traducirlo y enviarlo a motores como PostgreSQL, MySQL, Oracle o SQLite.

**SQLite y el concepto "In-Memory":**
SQLite es un motor de base de datos ligero que no requiere la instalación de un servidor complejo. El instructor utiliza la cadena de conexión `sqlite:///:memory:`, lo que crea una base de datos que vive exclusivamente en la memoria RAM. Es ultra-rápida para análisis temporales, pero es **volátil**: al cerrar el programa, los datos desaparecen.

### 🚀 Arquitectura Senior: Base de Datos Persistente (SQLite en Disco)

**El Desafío de Negocio:**
El método `sqlite:///:memory:` (utilizado en el curso) es excelente para pruebas rápidas (Sandbox), pero es **volátil**. Al apagar el servidor de Google Colab, la base de datos completa y todas sus tablas se borran. En un entorno de producción o análisis continuo, necesitamos que los datos sobrevivan a la sesión actual.

**La Solución (Persistencia en Disco):**
Para lograr que la base de datos sea permanente, modificamos la cadena de conexión del *Engine* de SQLAlchemy. En lugar de apuntar a la memoria RAM, le damos una ruta física absoluta hacia un archivo `.db` o `.sqlite` (ej. en nuestro Google Drive). Si el archivo no existe, SQLAlchemy lo creará automáticamente; si ya existe, se conectará a él, permitiéndonos consultar datos cargados días o meses atrás.

In [12]:
# -------------------------------------------------------------------
# OPCIÓN 2: CONEXIÓN PERSISTENTE (MODO PRODUCCIÓN PYME)
# NOTA: Este bloque es documental. El ejercicio del curso usa :memory:
# -------------------------------------------------------------------
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, inspect, text

# 1. Definimos la ruta física donde vivirá nuestra base de datos (Ej. Google Drive)
# El prefijo "sqlite:///" indica el dialecto. Las barras adicionales indican una ruta absoluta.
RUTA_DB_FISICA = 'sqlite:////content/drive/MyDrive/Pandas/banco_clientes_produccion.db'

# 2. Creamos el Motor de Conexión (Engine) apuntando al archivo físico
# echo=True (Opcional) permite ver en la consola el código SQL real que SQLAlchemy genera por debajo
engine_persistente = create_engine(RUTA_DB_FISICA, echo=False)

print(f"✅ Motor de Base de Datos conectado exitosamente al disco en: {RUTA_DB_FISICA}")
print("Los datos cargados aquí sobrevivirán al cierre de la sesión.")

✅ Motor de Base de Datos conectado exitosamente al disco en: sqlite:////content/drive/MyDrive/Pandas/banco_clientes_produccion.db
Los datos cargados aquí sobrevivirán al cierre de la sesión.


### ⚠️ Riesgos y Desventajas del Enfoque Persistente

**Consideraciones Críticas de Arquitectura:**
Al trabajar con bases de datos físicas desde Pandas, debemos implementar controles más estrictos en nuestro código:

1. **Riesgo de Sobrescritura (`if_exists`):** Si ejecutamos el bloque de creación/carga de la tabla (ej. `to_sql`) dos veces por error, podemos duplicar todos los registros de los clientes o sobrescribir la tabla existente si no configuramos correctamente el parámetro `if_exists='append'` (agregar) o `if_exists='replace'` (reemplazar).
2. **Latencia de I/O (Input/Output):** Escribir y leer desde un disco duro (o desde Google Drive) es considerablemente más lento que operar en la memoria RAM ultrarrápida. Para bases de datos gigantes, este cuello de botella se hace notar.
3. **Bloqueos de Conexión (Locks):** SQLite está diseñado para uso local. Si dos usuarios o dos scripts de Python intentan escribir en el archivo `.db` físico exactamente al mismo tiempo, la base de datos se bloqueará para proteger la integridad, lanzando un error de concurrencia. (Para sistemas multiusuario reales, se debe migrar a PostgreSQL o MySQL).

### 🧠 Fundamentos de Persistencia: Bases de Datos y el Patrón ORM

**Concepto de Negocio (Relacional vs. NoSQL):**
El motor de almacenamiento debe elegirse según la naturaleza de la información que la empresa maneja:
* **Bases de Datos Relacionales (SQL):** Almacenan información en tablas rígidamente estructuradas, conectadas por llaves (IDs). Son el estándar obligatorio para transacciones, contabilidad y facturación, ya que garantizan la integridad referencial (ej. no puedes borrar un cliente si tiene deudas activas). Ejemplos: PostgreSQL, MySQL, SQLite.
* **Bases de Datos No Relacionales (NoSQL):** Estructuras flexibles (basadas en documentos como JSON, o llave-valor). Ideales para información menos estricta, como el catálogo de características variables de un producto o el registro de clics de una página web. Ejemplos: MongoDB, Redis.



**La Arquitectura de SQLAlchemy (El ORM):**
SQLAlchemy implementa el patrón de Mapeo Objeto-Relacional (ORM). Esta capa de abstracción traduce las tablas de la base de datos SQL en **Clases** y los registros en **Objetos** nativos de Python.
Esto nos permite interactuar con la base de datos aplicando los principios de la Programación Orientada a Objetos (POO), evitando escribir sentencias SQL en texto plano, reduciendo errores de seguridad (como la inyección SQL) y facilitando el mantenimiento del código (*Clean Code*).

📚 **Referencia Técnica:** Para explorar la creación de modelos de datos avanzados y el manejo de sesiones corporativas, se recomienda consultar la [Documentación oficial de SQLAlchemy](https://www.sqlalchemy.org/).

##**Escribiendo en un banco de datos**

In [5]:
import pandas as pd

In [6]:
archivo = '/content/drive/MyDrive/Pandas/clientes_banco.csv'

In [7]:
datos = pd.read_csv(archivo)

In [8]:
datos.head()

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


In [9]:
datos.to_sql('clientes', engine, index=False)

438463

In [10]:
inspector = inspect(engine)

In [11]:
print(inspector.get_table_names())

['clientes']


### 🏗️ Arquitectura de Ingesta: Estrategias de Carga SQL (`if_exists`)

**El Desafío de Negocio:**
Al automatizar la ingesta de datos hacia una base de datos SQL con `to_sql()`, el sistema debe saber exactamente qué hacer si detecta que la tabla destino ya existe (por ejemplo, si el script se ejecuta dos veces por error, o si es una carga programada diaria).

**Matriz de Decisión (Parámetro `if_exists`):**

1. **Modo Refresco Total (`if_exists='replace'`):**
   * **Lógica:** Ejecuta un `DROP TABLE` (borra la tabla antigua por completo) y crea una nueva con los datos del DataFrame actual.
   * **Caso de Uso Habitual:** Tablas maestras o catálogos (ej. Lista de Precios actualizada, Catálogo de Productos). Si el proveedor envía un Excel nuevo cada mes, queremos borrar el viejo y quedarnos solo con la foto actual.

2. **Modo Incremental (`if_exists='append'`):**
   * **Lógica:** Mantiene la tabla y su estructura original, insertando las nuevas filas al final (como un `INSERT INTO`).
   * **Caso de Uso Habitual:** Datos transaccionales o históricos (ej. Ventas del día, Registros de auditoría, Nuevos clientes registrados hoy). No queremos borrar la historia, solo sumar el movimiento diario.

3. **Modo Estricto / Bloqueo (`if_exists='fail'`):**
   * **Lógica:** Comportamiento por defecto. Si la tabla ya existe, detiene la ejecución del programa y lanza un error (`ValueError`).
   * **Caso de Uso Habitual:** Migraciones iniciales de bases de datos o inyección de datos críticos (ej. Saldos iniciales contables). Garantiza que una tabla sensible jamás sea alterada por una ejecución accidental del script.

In [13]:
import pandas as pd
from sqlalchemy import inspect

# 1. Definición de la Fuente (El archivo CSV en tu Drive)
RUTA_CSV_CLIENTES = '/content/drive/MyDrive/Pandas/clientes_banco.csv'

# 2. Fase de Extracción (Leemos el archivo plano a la memoria de Pandas)
df_clientes = pd.read_csv(RUTA_CSV_CLIENTES)
print(f"📥 Archivo CSV extraído. Total de registros en memoria: {len(df_clientes)}")

# 3. Fase de Carga (Inyección SQL hacia la Base de Datos)
# Asumimos que la variable 'engine_persistente' fue creada en el paso anterior.
# Usamos 'replace' para que el Colab no falle si ejecutamos la celda varias veces.
NOMBRE_TABLA = 'clientes'

df_clientes.to_sql(
    name=NOMBRE_TABLA,
    con=engine_persistente,
    if_exists='replace',
    index=False
)

print(f"✅ Migración completada. Datos transferidos a la tabla SQL: '{NOMBRE_TABLA}'.")

# -------------------------------------------------------------------
# 4. Auditoría de Estructura (Control de Calidad QA con el Inspector)
# -------------------------------------------------------------------
print("\n--- Iniciando Auditoría de Catálogo SQL ---")

# Creamos el objeto inspector conectándolo a nuestro motor
auditor_sql = inspect(engine_persistente)

# Solicitamos la lista de tablas existentes en el motor
tablas_existentes = auditor_sql.get_table_names()

print(f"📋 Tablas detectadas en el motor de Base de Datos: {tablas_existentes}")

# Verificación lógica automatizada
if NOMBRE_TABLA in tablas_existentes:
    print(f"🟢 Estado de Salud: APROBADO. La tabla '{NOMBRE_TABLA}' existe en el esquema.")
else:
    print(f"🔴 ALERTA: La tabla '{NOMBRE_TABLA}' no fue encontrada en la base de datos.")

📥 Archivo CSV extraído. Total de registros en memoria: 438463
✅ Migración completada. Datos transferidos a la tabla SQL: 'clientes'.

--- Iniciando Auditoría de Catálogo SQL ---
📋 Tablas detectadas en el motor de Base de Datos: ['clientes']
🟢 Estado de Salud: APROBADO. La tabla 'clientes' existe en el esquema.


##**Leyendo una consulta SQL**

### 🗣️ Fundamentos de SQL: El Lenguaje de Negocios

**Concepto Gerencial:**
SQL (Structured Query Language) no es un lenguaje de programación tradicional; es un **lenguaje de consultas**. Funciona exactamente como las instrucciones que un gerente le daría a un analista de datos: *"Tráeme estas columnas, de este archivo, pero solo de los clientes que cumplan esta condición"*.

**Anatomía de una Consulta (Query) y sus Cláusulas:**
Para construir una pregunta estructurada, utilizamos bloques lógicos (cláusulas). El orden de escritura es estricto:

* **`SELECT` (¿Qué quiero ver?):** Especifica las columnas exactas que necesitamos. Evita traer datos basura a la memoria. (Ej. `SELECT nombre, cuit, saldo_deudor`).
* **`FROM` (¿De dónde lo saco?):** Indica la tabla o "archivo maestro" a consultar. (Ej. `FROM tabla_clientes`).
* **`WHERE` (¿Cuál es el filtro?):** Aplica la lógica de negocio para excluir lo que no sirve. (Ej. `WHERE saldo_deudor > 0`).
* **`ORDER BY` (¿Cómo lo presento?):** Ordena el resultado de forma ascendente (ASC) o descendente (DESC). (Ej. `ORDER BY saldo_deudor DESC` para ver a los mayores deudores primero).
* **`GROUP BY` (¿Cómo lo totalizo?):** Agrupa filas que tienen los mismos valores para calcular subtotales. (Ej. Agrupar por `vendedor_id` para sumar comisiones).
* **`LIMIT` (¿Cuántos necesito?):** Restringe el volumen de respuesta. (Ej. `LIMIT 10` para un Top 10).

**Ejemplo Práctico de Negocio:**
Si queremos buscar en el sistema de Recursos Humanos a nuestro equipo comercial para liquidar comisiones, la sentencia SQL cruda se vería así:

```sql
SELECT nombre, apellido, salario
FROM empleados
WHERE departamento = 'ventas';
```

In [14]:
query = 'SELECT * FROM clientes WHERE Categoria_de_renta = "Empleado"'

In [15]:
empleados = pd.read_sql(sql=text(query), con=engine.connect())

In [16]:
empleados

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008815,46,Nivel superior,Casado,2,Empleado,Contabilidad,2,270000.0,1,Casa/Departamento propio
4,5112956,46,Nivel superior,Casado,2,Empleado,Contabilidad,2,270000.0,1,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
226054,6837905,43,Nivel intermedio,Casado,3,Empleado,Otro,7,355050.0,1,Casa/Departamento propio
226055,6837906,43,Nivel intermedio,Casado,3,Empleado,Otro,7,355050.0,1,Casa/Departamento propio
226056,6839936,34,Nivel intermedio,Casado,3,Empleado,Construcción Civil,5,135000.0,1,Casa/Departamento propio
226057,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio


In [17]:
empleados.to_sql('empleados', con=engine.connect(), index=False)

226059

In [18]:
# Cuando queremos leer una tabla completa (pero no un query) read_sql_table
pd.read_sql_table('empleados', con=engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008815,46,Nivel superior,Casado,2,Empleado,Contabilidad,2,270000.0,1,Casa/Departamento propio
4,5112956,46,Nivel superior,Casado,2,Empleado,Contabilidad,2,270000.0,1,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
226054,6837905,43,Nivel intermedio,Casado,3,Empleado,Otro,7,355050.0,1,Casa/Departamento propio
226055,6837906,43,Nivel intermedio,Casado,3,Empleado,Otro,7,355050.0,1,Casa/Departamento propio
226056,6839936,34,Nivel intermedio,Casado,3,Empleado,Construcción Civil,5,135000.0,1,Casa/Departamento propio
226057,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio


In [19]:
#En este caso solo leemos algunas columnas
pd.read_sql_table('empleados', con=engine.connect(), columns=['ID_Cliente', 'Grado_estudio','Rendimiento_anual'])

,ID_Cliente,Grado_estudio,Rendimiento_anual
0,5008804,Nivel superior,427500.0
1,5008805,Nivel superior,427500.0
2,5008806,Nivel intermedio,112500.0
3,5008815,Nivel superior,270000.0
4,5112956,Nivel superior,270000.0
...,...,...,...
226054,6837905,Nivel intermedio,355050.0
226055,6837906,Nivel intermedio,355050.0
226056,6839936,Nivel intermedio,135000.0
226057,6840222,Nivel intermedio,103500.0


### 🔍 Interrogando a la Base de Datos (Queries SQL)

**Concepto de Negocio:**
No siempre necesitamos descargar toda la base de datos a la memoria RAM de Python. El poder de SQL radica en que el motor de la base de datos hace el trabajo pesado de filtrar y agrupar la información *antes* de enviárnosla.

**Mejores Prácticas (Manejo de Conexiones):**
En entornos de producción, las conexiones a bases de datos son recursos limitados. En lugar de abrir una conexión flotante (`engine.connect()`), utilizamos el bloque `with engine.connect() as conexion:`. Esto garantiza que el "canal de comunicación" se cierre automáticamente apenas Pandas termine de descargar los datos, previniendo bloqueos o saturación del servidor.

**La Herramienta `text()`:**
Las versiones modernas de SQLAlchemy exigen que las consultas SQL en texto plano sean envueltas en la función `text()`. Esto es una medida de seguridad avanzada para prevenir inyecciones SQL y asegurar que el motor interprete correctamente la sintaxis.

In [20]:
import pandas as pd
from sqlalchemy import text

# 1. Diseñamos la pregunta de negocio en lenguaje SQL crudo
pregunta_sql = """
    SELECT * FROM clientes
    WHERE Categoria_de_renta = 'Empleado'
"""

# 2. Ejecutamos la consulta usando un "Context Manager" (Clean Code)
# Esto asegura que la conexión se abra y se cierre de forma segura
with engine_persistente.connect() as conexion_segura:

    # Usamos read_sql() porque le estamos pasando una Query, no un nombre de tabla
    df_empleados = pd.read_sql(
        sql=text(pregunta_sql),
        con=conexion_segura
    )

print(f"📊 Análisis completado. Se encontraron {len(df_empleados)} clientes clasificados como 'Empleado'.")
display(df_empleados.head(3))

📊 Análisis completado. Se encontraron 226059 clientes clasificados como 'Empleado'.


,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio


### 💾 Creación de Data Marts y Lectura Directa de Tablas

**Concepto de Negocio (Aislamiento de Datos):**
Una vez que hemos procesado un filtro complejo mediante una Query (ej. aislar solo a los empleados), es una buena práctica guardar ese resultado en una nueva tabla independiente. Esto permite que otros sectores de la empresa consulten directamente este subconjunto sin sobrecargar la tabla maestra original.

**Diferencia Crítica de Lectura:**
* `read_sql(Query)`: Se usa para hacerle una pregunta específica a la base de datos.
* `read_sql_table('Nombre')`: Se usa para descargar una tabla completa. Es más eficiente si sabemos que necesitamos el archivo entero, e incluye el parámetro `columns=[]` para descargar solo columnas puntuales sin necesidad de escribir código SQL (SELECT).

In [21]:
# -------------------------------------------------------------------
# BLOQUE 2: FASE DE CARGA (Guardando el Data Mart)
# -------------------------------------------------------------------
NOMBRE_NUEVA_TABLA = 'empleados'

# Guardamos el DataFrame filtrado como una nueva tabla física en el motor
df_empleados.to_sql(
    name=NOMBRE_NUEVA_TABLA,
    con=engine_persistente,
    if_exists='replace', # Reemplazamos si ya existe para evitar errores en pruebas
    index=False
)
print(f"✅ Data Mart creado. Tabla '{NOMBRE_NUEVA_TABLA}' guardada en la base de datos.")

# -------------------------------------------------------------------
# BLOQUE 3: LECTURA DIRECTA Y OPTIMIZADA (Columnas Específicas)
# -------------------------------------------------------------------
# Usamos el Context Manager (Clean Code) para abrir y cerrar la conexión
with engine_persistente.connect() as conexion_segura:

    # En lugar de escribir una Query SQL (SELECT x, y FROM z),
    # usamos read_sql_table y le pasamos una lista de columnas requeridas
    df_empleados_resumen = pd.read_sql_table(
        table_name=NOMBRE_NUEVA_TABLA,
        con=conexion_segura,
        columns=['ID_Cliente', 'Grado_estudio', 'Rendimiento_anual']
    )

print("\n📊 Lectura optimizada completada (Solo 3 columnas requeridas):")
display(df_empleados_resumen.head(3))

✅ Data Mart creado. Tabla 'empleados' guardada en la base de datos.

📊 Lectura optimizada completada (Solo 3 columnas requeridas):


,ID_Cliente,Grado_estudio,Rendimiento_anual
0,5008804,Nivel superior,427500.0
1,5008805,Nivel superior,427500.0
2,5008806,Nivel intermedio,112500.0


##**Actualizando un banco de datos**

In [22]:
query = 'SELECT * FROM clientes'

In [23]:
pd.read_sql(sql=text(query), con=engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,135000.0,0,Casa/Departamento propio
438459,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio
438460,6841878,22,Nivel superior,Soltero,1,Business Partner,Ventas,1,54000.0,0,Vive con los padres
438461,6842765,59,Nivel intermedio,Casado,2,Pensionista,Otro,0,72000.0,0,Casa/Departamento propio


In [24]:
#Borrando registros
#la biblioteca que importamos nos permite identificar los errores de ejecuciones en SQL
from sqlalchemy.exc import SQLAlchemyError
query = "DELETE FROM clientes WHERE ID_Cliente=5008804"
try:
  r_set=engine.connect().execute(text(query))
# en este except tenemos los errores contenidos en la biblioteca
except SQLAlchemyError as e:
  print(e)
else:
  print("#Registros borrados: ",r_set.rowcount)

#Registros borrados:  1


In [25]:
pd.read_sql_table('clientes', con=engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
2,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
3,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008810,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
438457,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,135000.0,0,Casa/Departamento propio
438458,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio
438459,6841878,22,Nivel superior,Soltero,1,Business Partner,Ventas,1,54000.0,0,Vive con los padres
438460,6842765,59,Nivel intermedio,Casado,2,Pensionista,Otro,0,72000.0,0,Casa/Departamento propio


In [26]:
#Actualizando registros
query = 'UPDATE clientes SET Grado_estudio="Nivel superior" WHERE ID_Cliente=5008808'
try:
  r_set=engine.connect().execute(text(query))
except SQLAlchemyError as e:
  print(e)
else:
  print("#Registros actualizados: ",r_set.rowcount)

#Registros actualizados:  1


In [27]:
pd.read_sql_table('clientes', con=engine.connect())

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
2,5008808,52,Nivel superior,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
3,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008810,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
438457,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,135000.0,0,Casa/Departamento propio
438458,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio
438459,6841878,22,Nivel superior,Soltero,1,Business Partner,Ventas,1,54000.0,0,Vive con los padres
438460,6842765,59,Nivel intermedio,Casado,2,Pensionista,Otro,0,72000.0,0,Casa/Departamento propio


In [28]:
inspector = inspect(engine)
print(inspector.get_table_names())

['clientes', 'empleados']


### ⚠️ Operaciones DML Avanzadas: UPDATE y DELETE

**Concepto de Negocio:**
A diferencia del `SELECT` (que solo lee datos), las operaciones `UPDATE` (modificar) y `DELETE` (borrar) alteran físicamente la base de datos. En un sistema corporativo, estas operaciones conllevan un riesgo inmenso: si olvidamos la cláusula `WHERE`, podríamos borrar o sobrescribir toda la tabla de clientes en un milisegundo.

**El Patrón de Seguridad (Transacciones y Try-Except):**
1. **Manejo de Errores (`try...except`):** Envolvemos la ejecución en este bloque. Si la consulta SQL tiene un error de sintaxis o viola una regla de la base de datos, el programa no colapsa, sino que captura el error y nos informa.
2. **El Commit Explícito:** Modificar datos no es automático. Debemos invocar `conexion.commit()` para confirmar los cambios definitivamente ("guardar"), asegurando el cumplimiento de las propiedades ACID de la base de datos.

In [29]:
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError

# 1. Definimos el ID del cliente que se dio de baja
cliente_a_borrar = 5008804

# 2. Diseñamos la Query DML (Con cláusula WHERE obligatoria)
query_delete = f"""
    DELETE FROM clientes
    WHERE ID_Cliente = {cliente_a_borrar}
"""

print(f"🗑️ Intentando borrar al cliente ID: {cliente_a_borrar}...")

# 3. Ejecutamos usando Try-Except y Context Manager
try:
    with engine_persistente.connect() as conexion:
        # Ejecutamos la consulta envuelta en text()
        resultado = conexion.execute(text(query_delete))

        # OBLIGATORIO: Confirmamos la transacción para guardar los cambios en disco
        conexion.commit()

        # Verificamos cuántas filas fueron afectadas
        print(f"✅ Operación exitosa. Registros eliminados: {resultado.rowcount}")

except SQLAlchemyError as e:
    print(f"🔴 Error crítico en la Base de Datos: {e}")

🗑️ Intentando borrar al cliente ID: 5008804...
✅ Operación exitosa. Registros eliminados: 1


In [30]:
# 1. Definimos el ID del cliente y su nuevo grado de estudio
cliente_a_actualizar = 5008808
nuevo_grado = 'Nivel superior'

# 2. Diseñamos la Query de Actualización
query_update = f"""
    UPDATE clientes
    SET Grado_estudio = '{nuevo_grado}'
    WHERE ID_Cliente = {cliente_a_actualizar}
"""

print(f"🔄 Intentando actualizar grado de estudio para el cliente ID: {cliente_a_actualizar}...")

# 3. Ejecutamos usando Try-Except y Context Manager
try:
    with engine_persistente.connect() as conexion:
        resultado = conexion.execute(text(query_update))

        # Confirmamos la transacción
        conexion.commit()

        print(f"✅ Operación exitosa. Registros actualizados: {resultado.rowcount}")

except SQLAlchemyError as e:
    print(f"🔴 Error crítico en la Base de Datos: {e}")

# -------------------------------------------------------------------
# 4. Auditoría de Verificación (Comprobamos los cambios leyendo la tabla)
# -------------------------------------------------------------------
# Leemos los primeros registros de la tabla para confirmar visualmente
with engine_persistente.connect() as conexion:
    df_verificacion = pd.read_sql_table('clientes', con=conexion)

print("\n📊 Vista previa de la tabla actualizada:")
display(df_verificacion.head(5))

🔄 Intentando actualizar grado de estudio para el cliente ID: 5008808...
✅ Operación exitosa. Registros actualizados: 1

📊 Vista previa de la tabla actualizada:


,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
2,5008808,52,Nivel superior,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
3,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008810,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio


# Desafio Final


### 🏆 Desafío Final: Operaciones CRUD en Base de Datos Bancaria

**El Requerimiento de Negocio:**
Se nos ha asignado la responsabilidad de administrar el banco de datos local de una institución financiera. Partiendo de un archivo maestro en formato CSV, debemos inicializar el motor SQL y ejecutar una serie de operaciones transaccionales críticas.

**Misión y Entregables:**
1. **Inicialización (Carga):** Crear la base de datos local con `SQLAlchemy` e inyectar los datos del archivo `clientes_banco.csv`.
2. **Operación UPDATE (Actualización):** Modificar el registro del cliente `ID 6840104`, ajustando su rendimiento anual a `300000`.
3. **Operación DELETE (Baja):** Eliminar el registro del cliente `ID 5008809` por cierre de cuenta.
4. **Operación INSERT (Alta):** Ingresar un nuevo cliente al sistema con las siguientes especificaciones:
   * `ID_cliente`: 6850985
   * `Edad`: 33
   * `Grado_estudio`: Doctorado
   * `Estado_civil`: Soltero
   * `Tamaño_familia`: 1
   * `Categoria_de_renta`: Empleado
   * `Ocupacion`: TI
   * `Años_empleado`: 2
   * `Rendimiento_anual`: 290000
   * `Tiene_carro`: 0
   * `Vivienda`: Casa/Departamento propio

*Nota Técnica para el Alta (INSERT):* La sintaxis requerida es `INSERT INTO tabla (columna1, columna2) VALUES (valor1, valor2)`. El orden de los valores debe mapear exactamente con el orden de las columnas declaradas.

In [32]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# ===================================================================
# FASE 1: INICIALIZACIÓN Y CARGA (ETL)
# ===================================================================
# Creamos el motor en memoria como pide el desafío
engine_desafio = create_engine('sqlite:///:memory:')

# Cargamos el CSV (Asegúrate de que esta ruta coincida con tu Drive)
ruta_csv = '/content/drive/MyDrive/Pandas/clientes_banco.csv'
datos_clientes = pd.read_csv(ruta_csv)

# Inyectamos los datos a la tabla SQL
datos_clientes.to_sql('clientes', con=engine_desafio, index=False)
print("✅ Fase 1: Base de datos inicializada y tabla 'clientes' cargada.")

# ===================================================================
# FASE 2: PREPARACIÓN DE LAS QUERIES (Lógica de Negocio)
# ===================================================================
# 1. UPDATE: Actualizar rendimiento anual
query_update = text("""
    UPDATE clientes
    SET Rendimiento_anual = 300000.0
    WHERE ID_Cliente = 6840104
""")

# 2. DELETE: Dar de baja a un cliente
query_delete = text("""
    DELETE FROM clientes
    WHERE ID_Cliente = 5008809
""")

# 3. INSERT: Dar de alta a un nuevo cliente
query_insert = text("""
    INSERT INTO clientes (
        ID_Cliente, Edad, Grado_estudio, Estado_civil, Tamaño_familia,
        Categoria_de_renta, Ocupacion, Años_empleado, Rendimiento_anual,
        Tiene_carro, Vivienda
    )
    VALUES (
        6850985, 33, 'Doctorado', 'Soltero', 1,
        'Empleado', 'TI', 2, 290000, 0, 'Casa/Departamento propio'
    )
""")

# ===================================================================
# FASE 3: EJECUCIÓN TRANSACCIONAL (Clean Code / Arquitectura Senior)
# ===================================================================
print("\n--- Iniciando Transacciones DML ---")

try:
    # Abrimos la conexión segura con el Context Manager
    with engine_desafio.connect() as conexion:

        # Ejecutamos las tres operaciones de escritura en la misma sesión
        res_update = conexion.execute(query_update)
        print(f"🔄 UPDATE exitoso. Registros modificados: {res_update.rowcount}")

        res_delete = conexion.execute(query_delete)
        print(f"🗑️ DELETE exitoso. Registros eliminados: {res_delete.rowcount}")

        res_insert = conexion.execute(query_insert)
        print(f"➕ INSERT exitoso. Registros creados: {res_insert.rowcount}")

        # FIRMA DEL CONTRATO: Confirmamos todos los cambios en la base de datos
        conexion.commit()
        print("🟢 Transacción completada: Todos los cambios han sido guardados (Commit).")

except SQLAlchemyError as error:
    print(f"🔴 Error crítico en la transacción. Se cancelan las operaciones: {error}")

# ===================================================================
# FASE 4: AUDITORÍA DE RESULTADOS
# ===================================================================
# Verificamos específicamente al nuevo cliente para confirmar el Alta
with engine_desafio.connect() as conexion:
    df_auditoria = pd.read_sql(
        sql=text("SELECT * FROM clientes WHERE ID_Cliente = 6850985"),
        con=conexion
    )

print("\n📊 Auditoría: Vista del nuevo cliente ingresado:")
display(df_auditoria)

✅ Fase 1: Base de datos inicializada y tabla 'clientes' cargada.

--- Iniciando Transacciones DML ---
🔄 UPDATE exitoso. Registros modificados: 1
🗑️ DELETE exitoso. Registros eliminados: 1
➕ INSERT exitoso. Registros creados: 1
🟢 Transacción completada: Todos los cambios han sido guardados (Commit).

📊 Auditoría: Vista del nuevo cliente ingresado:


,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,6850985,33,Doctorado,Soltero,1,Empleado,TI,2,290000.0,0,Casa/Departamento propio
